## Environment Setting
Google drive mount (for Colab users) and package importing.
You can optionally install and import torchensemble package for ensemble learning

In [2]:
# For Colab users
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import sys
sys.path.insert(0,'/content/drive/MyDrive/Project')

Mounted at /content/drive/


In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import random  #
import argparse  #

import torchvision.transforms as transforms  #
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from data_utils import Mydataset, Mytensordataset, collate_fn
from models import ConvLSTM

In [ ]:
import importlib
importlib.reload(sys.modules['modules'])
importlib.reload(sys.modules['models'])

<module 'models' from '/content/drive/MyDrive/Project/models.py'>

In [4]:
!pip install git+https://github.com/snuml2021tmp/Ensemble-pytorch.git

  Cloning https://github.com/snuml2021tmp/Ensemble-pytorch.git to /tmp/pip-req-build-r2izmns7
  Running command git clone -q https://github.com/snuml2021tmp/Ensemble-pytorch.git /tmp/pip-req-build-r2izmns7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/a8/eb/a48f25c967526b66d5f1fa7a984594f0bf0a5afafa94a8c4dbc317744620/scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/12/ec3f2e203afa394a149911729357aa48affc59c20e2c1c8297a60f33f133/threadpoolctl-2.1.0-py3-none-any.whl
  Created wheel for torchensemble: filename=torchensemble-0.1.4-cp37-none-any.whl size=42356 sha256=e332484d6beb5c5ef2bd30b9f7178dc14dcc67119ca6ce5caf32afe3c26d85cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-7e9brlk2/wheels/c4/7c/f6/783f62cb52be1d42975ef52549318529c50041cd2ae3bd3a6a
Successfully built torchensemble
  Found

In [5]:
# Only For Ensemble
from torchensemble import GradientBoostingClassifier
from torchensemble import SoftGradientBoostingClassifier
from torchensemble import VotingClassifier
from torchensemble import BaggingClassifier
from torchensemble import FusionClassifier
from torchensemble import SnapshotEnsembleClassifier
from torchensemble.utils.logging import set_logger
from torchensemble.utils import io

## (Optional) Sample Visualization
You can see actual sample images and sorted class indices. Additional matplotlib package is needed.

In [ ]:
# Just for reference: see actual samples
import matplotlib.pyplot as plt

alphabet = {
        'A(a)' : '0', 
        'B(b)' : '1', 
        'C(c)' : '2', 
        'D(d)' : '3', 
        'E(e)' : '4', 
        'F(f)' : '5', 
        'G(g)' : '6', 
        'H(h)' : '7', 
        'I(i)' : '8', 
        'J(j)' : '9', 
        'K(k)' : '10', 
        'L(l)' : '11', 
        'M(m)' : '12', 
        'N(n)' : '13', 
        'O(o)' : '14', 
        'P(p)' : '15', 
        'Q(q)' : '16', 
        'R(r)' : '17', 
        'S(s)' : '18', 
        'T(t)' : '19', 
        'U(u)' : '20', 
        'V(v)' : '21', 
        'W(w)' : '22', 
        'X(x)' : '23', 
        'Y(y)' : '24', 
        'Z(z)' : '25'
    }

In [ ]:
# Just for reference: see actual samples
idx = 10
sample = np.load(f'./data/emnist/train/numpy/{idx}.npy')
sample_target = np.loadtxt('./data/emnist/train/label.txt')[idx]

plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(1, 10, i+1)
    ax = plt.gca()
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    plt.imshow(sample[i], cmap='gray')
    
plt.show()
print("sorted label: ", end=' ')
label_str = '('
for i in range(10):
    print(int(sample_target[i].item()), end=' ')
    label_str += " " + list(alphabet.keys())[int(sample_target[i].item())]
label_str += " )"
print()
print(label_str)

In [6]:
# Use 0th GPU for training
torch.cuda.set_device(0)

In [7]:
# fix random seed to increase reproducibility
# NOTE: Do not modify here!
SEQUENCE_LENGTH = 10
NUM_CLASSES = 26

random_seed = 7
torch.manual_seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
%env CUBLAS_WORKSPACE_CONFIG=:16:8

def seed_worker(worker_seed):
    np.random.seed(worker_seed)
    random.seed(worker_seed)

num_workers = 0

env: CUBLAS_WORKSPACE_CONFIG=:16:8


In [8]:
# NOTE: you can modify mean and std for normalization
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

In [9]:
print_interval = 15
max_epoch = 1
batch_size = 16

In [10]:
# NOTE: modify path for your setting

# Option 1: use Mydataset (both for local and Colab users)
#train_ds = Mydataset('./data/train/numpy', './data/train/label.txt', transform=transform)
#valid_ds = Mydataset('./data/valid/numpy', './data/valid/label.txt', False, transform=transform)

# Option 2: use Mytensordataset (Colab users should use this)
train_ds = Mytensordataset('./drive/MyDrive/Project/data/train/img.pt', './drive/MyDrive/Project/data/train/label.pt', True, transform=train_transform)
valid_ds = Mytensordataset('./drive/MyDrive/Project/data/valid/img.pt', './drive/MyDrive/Project/data/valid/label.pt', False, transform=valid_transform)
test_ds = Mytensordataset('./drive/MyDrive/Project/data/test/img.pt', './drive/MyDrive/Project/data/test/label.pt', False, transform=valid_transform)

train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, worker_init_fn=seed_worker, collate_fn=collate_fn, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, num_workers=num_workers, worker_init_fn=seed_worker, collate_fn=collate_fn, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, num_workers=num_workers, worker_init_fn=seed_worker, collate_fn=collate_fn, shuffle=True)

In [11]:
# You can add or modify your ConvLSTM's hyperparameter (keys and values)
kwargs = {
    'cnn_input_dim': 1,
    'cnn_hidden_size': 64,
    'rnn_hidden_size': 128,
    'rnn_num_layers': 3,
    'rnn_dropout': 0.5,
    'rnn_input_dim': 64,
    'batch_size': batch_size
}

In [12]:
# NOTE: you can freely modify or add training hyperparameters
print_interval = 625
max_epoch = 25

In [ ]:
# Non-ensemble learning
model = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
print(model)
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
model_optim = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.5e-3)
loss_func = nn.CrossEntropyLoss()

#scheduler = optim.lr_scheduler.CosineAnnealingLR(model_optim, max_epoch)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(model_optim, 'max', patience=0, factor = 0.5, verbose = True, min_lr = 1e-7)
# NOTE: you can define additional components
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
def train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, load_path=None, save_path='./model.pt'):
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    # Load your states
    loaded_epoch = 0
    loaded_best_acc = -1
    if load_path is not None:
        state = torch.load(load_path)
        model.load_state_dict(state["model"])
        model_optim.load_state_dict(state["optimizer"])
        loaded_epoch = state["epoch"]
        loaded_best_acc = state["best_acc"]
        # ...
        
    ##############################################################################
    #                          END OF YOUR CODE                                  #
    ##############################################################################
    
    best_valid_accuracy = 0 if loaded_best_acc == -1 else loaded_best_acc

    for epoch in np.array(list(range(max_epoch - loaded_epoch))) + loaded_epoch:
        n_samples = 0
        n_correct = 0
        model.train()
        for step, sample in enumerate(train_dl):
            img, label = sample  # (BxT, C=1, H, W), (BxT)
            img = img.cuda()
            label = label.cuda()
            outputs = model((img, label))
            ##############################################################################
            #                          IMPLEMENT YOUR CODE                               #
            ##############################################################################
            # Problem5: implement optimization part (about four short lines are sufficient)
            #label = torch.tensor(label, dtype=torch.float64).cuda()
            loss = loss_func(outputs, label)
            model_optim.zero_grad()
            loss.backward()
            model_optim.step()

            ##############################################################################
            #                          END OF YOUR CODE                                  #
            ##############################################################################
            n_samples += img.size(0)
            n_correct += (outputs.argmax(-1) == label).sum().item()
            if (step + 1) % print_interval == 0:
                print('epoch:', epoch + 1, 'step:', step + 1, 'loss:', loss.item(), 'accuracy:', 100 * (n_correct / n_samples))
                
        n_samples = 0
        n_correct = 0
        with torch.no_grad():
            model.eval()
            for step, sample in enumerate(valid_dl):
                img, label = sample
                img = img.cuda()
                outputs = model(img)
                pred = np.argmax(outputs.cpu().data.numpy(), axis=1)
                label = label.data.numpy()
                n_samples += label.shape[0]
                n_correct += (pred == label).astype(float).sum()
            valid_accuracy = 100 * (n_correct/n_samples)
            if valid_accuracy > best_valid_accuracy:
                print("New best valid accuracy, saving model")
                ##############################################################################
                #                          IMPLEMENT YOUR CODE                               #
                ##############################################################################
                # Save your states
                state = {
                    "model": model.state_dict(),
                    "optimizer": model_optim.state_dict(),
                    "epoch": epoch + 1,
                    "best_acc": best_valid_accuracy,
                    # ...
                }
                ##############################################################################
                #                          END OF YOUR CODE                                  #
                ##############################################################################
                torch.save(state, save_path)
                best_valid_accuracy = valid_accuracy
            print('Valid epoch: %d, Valid accuracy: %.2f, Best valid accuracy: %.2f' % (epoch + 1, valid_accuracy, best_valid_accuracy))
        
        scheduler.step(valid_accuracy)  

In [ ]:
def eval(valid_dl, load_path):
    state = torch.load(load_path)
    model.load_state_dict(state["model"])
    n_samples = 0
    n_correct = 0
    with torch.no_grad():
        model.eval()
        for step, sample in enumerate(valid_dl):
            img, label = sample
            img = img.cuda()
            outputs = model(img)
            pred = np.argmax(outputs.cpu().data.numpy(), axis=1)
            label = label.data.numpy()
            n_samples += label.shape[0]
            n_correct += (pred == label).astype(float).sum()
        valid_accuracy = 100 * (n_correct/n_samples)
        print('Valid accuracy: %.2f' % (valid_accuracy))

In [ ]:
load_path = None
train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, load_path=load_path, save_path='./model.pt')


In [ ]:
## load and evaluate non-ensemble model
load_path = './model.pt'
eval(test_dl, load_path)

In [ ]:
####### Optional: ENSEMBLE - model type / optimizer / scheduler ########
estimator = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
# set ensemble model, optimizer, (optionally) lr scheduler
model = VotingClassifier(
    estimator = estimator,
    n_estimators = 10
)
model.set_optimizer(
    "Adam",
    lr = 0.001,
    weight_decay=0.5e-3
)
# Note: learning rate scheduler is optional
model.set_scheduler(
  "CosineAnnealingLR",
   T_max = max_epoch
)
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
logger = set_logger("Start Logging", use_tb_logger=False)
# epoch, optimizers, scheduler, best_acc, est_idx = io.load(model, './', use_scheduler=True)  # For load-and-rerun
model.fit(
    train_dl,
    epochs=25,
    test_loader=valid_dl,
    save_dir = "/content/drive/MyDrive/Project/ensemble_16_2"
    # retrain=True,  # For load-and-rerun
    # loaded_optimizers=optimizers,  # For load-and-rerun
    # loaded_scheduler=scheduler,  # For load-and-rerun
    # loaded_epoch=epoch,  # For load-and-rerun
    # loaded_est_idx=est_idx,  # For load-and-rerun
    # loaded_best_acc=best_acc,  # For load-and-rerun
)
acc = model.evaluate(valid_dl)
print("Valid accuracy: %.2f" % acc)

Log will be saved in '/content/logs'.
Create folder 'logs/'
Start logging into file /content/logs/Start Logging-2021_06_22_02_10.log...
Estimator: 000 | Epoch: 000 | Batch: 000 | LR: 0.00100 | Loss: 3.25561 | Correct: 5/160
Estimator: 000 | Epoch: 000 | Batch: 100 | LR: 0.00100 | Loss: 2.56960 | Correct: 26/160
Estimator: 000 | Epoch: 000 | Batch: 200 | LR: 0.00100 | Loss: 2.42615 | Correct: 24/160
Estimator: 000 | Epoch: 000 | Batch: 300 | LR: 0.00100 | Loss: 2.50388 | Correct: 19/160
Estimator: 000 | Epoch: 000 | Batch: 400 | LR: 0.00100 | Loss: 2.36971 | Correct: 24/160
Estimator: 000 | Epoch: 000 | Batch: 500 | LR: 0.00100 | Loss: 2.35795 | Correct: 26/160
Estimator: 000 | Epoch: 000 | Batch: 600 | LR: 0.00100 | Loss: 2.32676 | Correct: 26/160
Estimator: 000 | Epoch: 000 | Batch: 700 | LR: 0.00100 | Loss: 2.22657 | Correct: 33/160
Estimator: 000 | Epoch: 000 | Batch: 800 | LR: 0.00100 | Loss: 2.26246 | Correct: 26/160
Estimator: 000 | Epoch: 000 | Batch: 900 | LR: 0.00100 | Loss: 2

2021-06-22 02:24:26,407 - INFO: Epoch: 000 | Validation Acc: 37.802 % | Historical Best: 37.802 %
2021-06-22 02:24:26,418 - INFO: Saving the model to `/content/drive/MyDrive/Project/ensemble_16_2/VotingClassifier_ConvLSTM_10_ckpt.pth`


Estimator: 000 | Epoch: 001 | Batch: 000 | LR: 0.00100 | Loss: 1.73502 | Correct: 61/160
Estimator: 000 | Epoch: 001 | Batch: 100 | LR: 0.00100 | Loss: 1.83996 | Correct: 44/160
Estimator: 000 | Epoch: 001 | Batch: 200 | LR: 0.00100 | Loss: 1.79786 | Correct: 45/160
Estimator: 000 | Epoch: 001 | Batch: 300 | LR: 0.00100 | Loss: 1.67892 | Correct: 48/160
Estimator: 000 | Epoch: 001 | Batch: 400 | LR: 0.00100 | Loss: 1.67881 | Correct: 59/160
Estimator: 000 | Epoch: 001 | Batch: 500 | LR: 0.00100 | Loss: 1.48190 | Correct: 67/160
Estimator: 000 | Epoch: 001 | Batch: 600 | LR: 0.00100 | Loss: 1.75370 | Correct: 57/160
Estimator: 000 | Epoch: 001 | Batch: 700 | LR: 0.00100 | Loss: 1.63443 | Correct: 56/160
Estimator: 000 | Epoch: 001 | Batch: 800 | LR: 0.00100 | Loss: 1.64244 | Correct: 53/160
Estimator: 000 | Epoch: 001 | Batch: 900 | LR: 0.00100 | Loss: 1.55835 | Correct: 69/160
Estimator: 000 | Epoch: 001 | Batch: 1000 | LR: 0.00100 | Loss: 1.63614 | Correct: 55/160
Estimator: 000 | Epo

2021-06-22 02:38:27,096 - INFO: Epoch: 001 | Validation Acc: 72.464 % | Historical Best: 72.464 %
2021-06-22 02:38:27,102 - INFO: Saving the model to `/content/drive/MyDrive/Project/ensemble_16_2/VotingClassifier_ConvLSTM_10_ckpt.pth`


Estimator: 000 | Epoch: 002 | Batch: 000 | LR: 0.00098 | Loss: 1.38040 | Correct: 78/160
Estimator: 000 | Epoch: 002 | Batch: 100 | LR: 0.00098 | Loss: 1.36697 | Correct: 73/160
Estimator: 000 | Epoch: 002 | Batch: 200 | LR: 0.00098 | Loss: 1.09299 | Correct: 84/160
Estimator: 000 | Epoch: 002 | Batch: 300 | LR: 0.00098 | Loss: 1.19201 | Correct: 92/160
Estimator: 000 | Epoch: 002 | Batch: 400 | LR: 0.00098 | Loss: 1.14306 | Correct: 90/160
Estimator: 000 | Epoch: 002 | Batch: 500 | LR: 0.00098 | Loss: 1.31628 | Correct: 90/160
Estimator: 000 | Epoch: 002 | Batch: 600 | LR: 0.00098 | Loss: 1.10388 | Correct: 91/160
Estimator: 000 | Epoch: 002 | Batch: 700 | LR: 0.00098 | Loss: 1.14651 | Correct: 93/160
Estimator: 000 | Epoch: 002 | Batch: 800 | LR: 0.00098 | Loss: 0.98383 | Correct: 98/160
Estimator: 000 | Epoch: 002 | Batch: 900 | LR: 0.00098 | Loss: 1.17467 | Correct: 93/160
Estimator: 000 | Epoch: 002 | Batch: 1000 | LR: 0.00098 | Loss: 0.97867 | Correct: 99/160
Estimator: 000 | Epo

...

Estimator: 000 | Epoch: 023 | Batch: 000 | LR: 0.00002 | Loss: 0.08114 | Correct: 156/160
Estimator: 000 | Epoch: 023 | Batch: 100 | LR: 0.00002 | Loss: 0.04103 | Correct: 160/160
Estimator: 000 | Epoch: 023 | Batch: 200 | LR: 0.00002 | Loss: 0.17572 | Correct: 149/160
Estimator: 000 | Epoch: 023 | Batch: 300 | LR: 0.00002 | Loss: 0.10874 | Correct: 156/160
Estimator: 000 | Epoch: 023 | Batch: 400 | LR: 0.00002 | Loss: 0.10270 | Correct: 154/160
Estimator: 000 | Epoch: 023 | Batch: 500 | LR: 0.00002 | Loss: 0.08489 | Correct: 157/160
Estimator: 000 | Epoch: 023 | Batch: 600 | LR: 0.00002 | Loss: 0.12292 | Correct: 152/160
Estimator: 000 | Epoch: 023 | Batch: 700 | LR: 0.00002 | Loss: 0.06940 | Correct: 157/160
Estimator: 000 | Epoch: 023 | Batch: 800 | LR: 0.00002 | Loss: 0.05717 | Correct: 159/160
Estimator: 000 | Epoch: 023 | Batch: 900 | LR: 0.00002 | Loss: 0.29207 | Correct: 148/160
Estimator: 000 | Epoch: 023 | Batch: 1000 | LR: 0.00002 | Loss: 0.19859 | Correct: 151/160
Estimator

2021-06-22 07:47:13,026 - INFO: Epoch: 023 | Validation Acc: 90.122 % | Historical Best: 90.218 %


Estimator: 000 | Epoch: 024 | Batch: 000 | LR: 0.00000 | Loss: 0.12716 | Correct: 153/160
Estimator: 000 | Epoch: 024 | Batch: 100 | LR: 0.00000 | Loss: 0.11384 | Correct: 157/160
Estimator: 000 | Epoch: 024 | Batch: 200 | LR: 0.00000 | Loss: 0.09217 | Correct: 155/160
Estimator: 000 | Epoch: 024 | Batch: 300 | LR: 0.00000 | Loss: 0.23230 | Correct: 150/160
Estimator: 000 | Epoch: 024 | Batch: 400 | LR: 0.00000 | Loss: 0.04746 | Correct: 159/160
Estimator: 000 | Epoch: 024 | Batch: 500 | LR: 0.00000 | Loss: 0.17284 | Correct: 151/160
Estimator: 000 | Epoch: 024 | Batch: 600 | LR: 0.00000 | Loss: 0.21146 | Correct: 151/160
Estimator: 000 | Epoch: 024 | Batch: 700 | LR: 0.00000 | Loss: 0.05165 | Correct: 159/160
Estimator: 000 | Epoch: 024 | Batch: 800 | LR: 0.00000 | Loss: 0.07260 | Correct: 155/160
Estimator: 000 | Epoch: 024 | Batch: 900 | LR: 0.00000 | Loss: 0.09992 | Correct: 157/160
Estimator: 000 | Epoch: 024 | Batch: 1000 | LR: 0.00000 | Loss: 0.05770 | Correct: 158/160
Estimator

2021-06-22 08:01:17,987 - INFO: Epoch: 024 | Validation Acc: 90.122 % | Historical Best: 90.218 %


Valid accuracy: 90.12


In [13]:
# load and evaluate historical best model ensemble model
estimator = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
# Note: set ensemble type, optimizer, and scheduler exactly same with saved model
model = VotingClassifier(estimator, n_estimators=10)
model.set_optimizer(
    "Adam",
    lr = 0.001,
    weight_decay = 0.5e-3
)
model.set_scheduler(
    "CosineAnnealingLR",
    T_max = max_epoch
)
_, _, _, _ = io.load(model, '/content/drive/MyDrive/Project/ensemble_16_2')

In [14]:
# I have tested with given test set.

model.evaluate(test_dl)

89.461